# 인구 데이터 기반 소득 예측 경진대회

이번 대회는 인구 데이터 바탕으로 소득이 5만달러 이하인지 초과인지 분류하는 대회입니다.

모델을 작성해서 성능을 확인해보고 하이퍼파라미터 튜닝에 따라서 어떻게 성능이 변하고 어떻게 접근해야 하는지 알아봅시다!

그럼 시작해볼까요?

## 데이터 불러오기
먼저 분석하려는 데이터를 작업장으로 가져오는 작업이 필요합니다.

이를 위해서 파이썬 라이브러리 중 하나인 Pandas를 이용합니다.

pandas 라이브러리는 엑셀과 같은 행과 열로 이루어진 테이블(table) 형식의 데이터를 편하게 다루도록 해주는 라이브러리입니다.

In [3]:
import pandas as pd  
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # FutureWarning 제거

# csv 형식으로 된 데이터 파일을 읽어옵니다.
train = pd.read_csv('data/train.csv')

train.head()

,id,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,target
0,0,32,Private,309513,Assoc-acdm,12,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,0
1,1,33,Private,205469,Some-college,10,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,United-States,1
2,2,46,Private,149949,Some-college,10,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,0
3,3,23,Private,193090,Bachelors,13,Never-married,Adm-clerical,Own-child,White,Female,0,0,30,United-States,0
4,4,55,Private,60193,HS-grad,9,Divorced,Adm-clerical,Not-in-family,White,Female,0,0,40,United-States,0


In [4]:
train.shape

(17480, 16)

총 16열과 17480행을 가진 데이터 입니다.

## 결측치 확인

결측치(NA: Not Available)란 값이 누락된 데이터를 말합니다.

보다 정확한 분석을 하기 위해서는 데이터의 결측치를 확인하고 적절히 처리해주어야 합니다.

이번 데이터에 결측치가 있나 확인해볼까요?

In [5]:
def check_missing_col(dataframe):
    missing_col = []
    for col in dataframe.columns:
        missing_values = sum(dataframe[col].isna())
        is_missing = True if missing_values >= 1 else False
        if is_missing:
            print(f'결측치가 있는 컬럼은: {col} 입니다')
            print(f'해당 컬럼에 총 {missing_values} 개의 결측치가 존재합니다.')
            missing_col.append([col, dataframe[col].dtype])
    if missing_col == []:
        print('결측치가 존재하지 않습니다')
    return missing_col

missing_col = check_missing_col(train)

결측치가 있는 컬럼은: workclass 입니다
해당 컬럼에 총 1836 개의 결측치가 존재합니다.
결측치가 있는 컬럼은: occupation 입니다
해당 컬럼에 총 1843 개의 결측치가 존재합니다.
결측치가 있는 컬럼은: native.country 입니다
해당 컬럼에 총 583 개의 결측치가 존재합니다.


결측치가 존재하는군요!

결측치 데이터가 범주형인지 수치형인지 unique() 메소드를 통하여 확인하겠습니다.

In [6]:
print(train['workclass'].unique())
print(train['occupation'].unique())
print(train['native.country'].unique())

['Private' 'State-gov' 'Local-gov' 'Self-emp-not-inc' 'Self-emp-inc'
 'Federal-gov' 'Without-pay' nan 'Never-worked']
['Craft-repair' 'Exec-managerial' 'Adm-clerical' 'Prof-specialty'
 'Machine-op-inspct' 'Other-service' 'Sales' 'Farming-fishing'
 'Transport-moving' 'Handlers-cleaners' 'Tech-support' 'Protective-serv'
 'Priv-house-serv' 'Armed-Forces' nan]
['United-States' 'Poland' 'Mexico' 'Ireland' 'Guatemala'
 'Dominican-Republic' 'Greece' 'El-Salvador' 'Portugal' 'Canada'
 'Philippines' 'India' 'Italy' 'England' 'Jamaica' 'Columbia' 'South'
 'Vietnam' 'Cuba' 'Laos' 'Hong' 'Haiti' 'Germany' 'Yugoslavia' 'Ecuador'
 'France' 'Puerto-Rico' 'Outlying-US(Guam-USVI-etc)' 'Taiwan' 'China'
 'Japan' 'Honduras' 'Peru' 'Nicaragua' 'Hungary' 'Cambodia' 'Iran'
 'Trinadad&Tobago' 'Thailand' 'Scotland' 'Holand-Netherlands' nan]


모두 데이터 값이 범주형이고

결측치들에 대해서 어떤 특별한 패턴은 보이지 않으므로

범주형 데이터에 대해서는 행을 삭제해주겠습니다.

In [7]:
# 결측치를 처리하는 함수를 작성합니다.
def handle_na(data, missing_col):
    temp = data.copy()
    for col, dtype in missing_col:
        if dtype == 'O':
            # 범주형 feature가 결측치인 경우 해당 행들을 삭제해 주었습니다.
            temp = temp.dropna(subset=[col])
    return temp

train = handle_na(train, missing_col)

# 결측치 처리가 잘 되었는지 확인해 줍니다.
missing_col = check_missing_col(train) 

결측치가 존재하지 않습니다


## 데이터 전처리

info() 메소드를 이용하여 데이터의 타입을 보면 상당히 많은 데이터가 범주형인 것을 확인 할 수 있습니다.

In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15081 entries, 0 to 15080
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              15081 non-null  int64 
 1   age             15081 non-null  int64 
 2   workclass       15081 non-null  object
 3   fnlwgt          15081 non-null  int64 
 4   education       15081 non-null  object
 5   education.num   15081 non-null  int64 
 6   marital.status  15081 non-null  object
 7   occupation      15081 non-null  object
 8   relationship    15081 non-null  object
 9   race            15081 non-null  object
 10  sex             15081 non-null  object
 11  capital.gain    15081 non-null  int64 
 12  capital.loss    15081 non-null  int64 
 13  hours.per.week  15081 non-null  int64 
 14  native.country  15081 non-null  object
 15  target          15081 non-null  int64 
dtypes: int64(8), object(8)
memory usage: 2.0+ MB


In [9]:
train.age

0        32
1        33
2        46
3        23
4        55
         ..
15076    35
15077    36
15078    50
15079    39
15080    33
Name: age, Length: 15081, dtype: int64

머신러닝 알고리즘은 문자열 데이터 속성을 입력받지 않으며 모든 데이터는 숫자형으로 표현되어야 합니다.

문자열로 구성된 범주형 속성 모두 숫자 값으로 변환/인코딩 하겠습니다.

적용할 인코딩 방식은 'Label Encoding' 입니다.

Label Encoding이란, 범주형 변수의 문자열을 수치형으로 변환하는 방법 중 하나입니다.

Label Encoding을 통해 숫자형으로 표현해보겠습니다.

In [10]:
#라벨인코딩을 하기 위함 dictionary map 생성 함수
def make_label_map(dataframe):
    label_maps = {}
    for col in dataframe.columns:
        if dataframe[col].dtype=='object':
            label_map = {'unknown':0}
            for i, key in enumerate(dataframe[col].unique()):
                label_map[key] = i  #새로 등장하는 유니크 값들에 대해 1부터 1씩 증가시켜 키값을 부여해줍니다.
            label_maps[col] = label_map
    return label_maps

# 각 범주형 변수에 인코딩 값을 부여하는 함수
def label_encoder(dataframe, label_map):
    for col in dataframe.columns:
        if dataframe[col].dtype=='object':
            dataframe[col] = dataframe[col].map(label_map[col])
            #dataframe[col] = dataframe[col].fillna(label_map[col]['unknown']) #혹시 모를 결측값은 unknown의 값(0)으로 채워줍니다.
    return dataframe

train = label_encoder(train, make_label_map(train))

다음과 같이 Label Encoding을 통해서 Gender에 해당하는 데이터들이 모두 숫자형으로 변환됨을 확인할 수 있습니다.



In [11]:
train.shape

(15081, 16)

In [12]:
train

,id,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,target
0,0,32,0,309513,0,12,0,0,0,0,0,0,0,40,0,0
1,1,33,0,205469,1,10,0,1,0,0,0,0,0,40,0,1
2,2,46,0,149949,1,10,0,0,0,0,0,0,0,40,0,0
3,3,23,0,193090,2,13,1,2,1,0,1,0,0,30,0,0
4,4,55,0,60193,3,9,2,2,2,0,1,0,0,40,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15076,15076,35,0,337286,7,14,1,1,2,2,0,0,0,40,0,0
15077,15077,36,0,182074,1,10,2,2,2,0,0,0,0,45,0,0
15078,15078,50,4,175070,4,15,0,3,0,0,0,0,0,45,0,1
15079,15079,39,0,202937,1,10,2,10,2,0,1,0,0,40,1,0


## 모델 선언과 학습

### train.csv 학습/검증셋 분리하기

모델링 후 성능을 검증하기 위해 학습셋과 검증셋으로 train 데이터를 나누어 보겠습니다.

In [13]:
from sklearn.model_selection import train_test_split

data = train.drop('id', axis = 1).copy() #필요없는 id열 삭제
train_data, val_data = train_test_split(data, test_size=0.5)
train_data.reset_index(inplace=True) #전처리 과정에서 데이터가 뒤섞이지 않도록 인덱스를 초기화
val_data.reset_index(inplace=True)

In [14]:
print( 'train 데이터 셋 모양 :', train_data.shape)
print( 'val 데이터 셋 모양 :', val_data.shape)

train 데이터 셋 모양 : (7540, 16)
val 데이터 셋 모양 : (7541, 16)


train 셋은 11310, val 셋은 3771 데이터로 나뉘어 진 것을 확인할 수 있습니다.

In [15]:
train_data.head()

,index,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,target
0,1891,17,0,148522,6,7,1,5,1,0,0,0,1721,15,0,0
1,2337,64,2,266080,3,9,2,2,2,0,1,0,0,35,0,0
2,650,45,0,148900,2,13,0,3,0,0,0,0,0,50,0,1
3,14938,40,0,287008,4,15,0,3,0,0,0,15024,0,55,22,1
4,11763,32,0,175878,8,4,0,0,0,0,0,0,0,35,0,0


이제는 모델 학습을 위해 train 데이터와 lable을 분리해보도록 하겠습니다.

In [16]:
X_train = train_data.drop(['index', 'target'], axis=1) #training 데이터에서 독립변수 추출
y_train = train_data.target #training 데이터에서 라벨 추출

In [17]:
X_train.shape

(7540, 14)

### 모델 학습

먼저, 1차로 제공해드린 베이스라인에서는 LogisticRegression 대해서 알아보았습니다. sklearn 패키지에서 제공하는 LogisticRegression 사용했었습니다.

이번 베이스라인에서는 대표적인 ML모델의 하나인 RandomForestClassifier 모델을 사용해봅시다.

Classifier 모델을 사용하는 이유는, 우리가 이번 대회에서 예측해야할 수입(target)이 분류 변수이기 때문입니다.

<br>
RandomForest 알고리즘에서 사용되는 의사 결정 트리는 feature 별 가지치기를 통해 데이터를 학습하는 알고리즘입니다.

의사 결정 나무에 대한 자세한 개념 설명은 데이콘 [오늘의 파이썬](https://dacon.io/competitions/open/235698/talkboard/403509?page=1&dtype=recent)을 참고해 주세요!

의사 결정 나무는 데이터를 쉽게 학습하는 알고리즘이지만 하나의 의사 결정 나무를 사용하는 것은 과적합(overfitting)의 문제를 발생 시킬 수 있습니다.

여기서 과적합(overfitting)이란 학습 데이터에 과하게 학습되어 새로운 데이터를 예측하지 못하는 문제를 말합니다.

쉽게 말해 우리가 공부를 할때 암기식으로 공부를 하여 새로운 문제를 해결하지 못하는 것 입니다!

<br>
RandomForest는 여러개의 의사 결정 나무를 활용하여 과적합의 문제를 해결합니다.

여러개의 의사 결정 트리를 활용하면 하나의 트리에 대한 의존도가 낮아집니다.

이는 하나의 트리가 과적합이 되어도 전체 모델은 과적합의 문제에서 벗어날 수 있음을 의미합니다.

sklearn 패키지를 이용하면 직접 RandomForest를 구현하지 않고 모델을 사용할 수 있습니다.

이번 베이스라인에서는 sklearn에서 제공하는 RandomForestClassifier 활용해보겠습니다.

In [19]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier() # 모델을 객체에 할당
model.fit(X_train, y_train) # 모델 학습

#pred = model.predict(test)

RandomForestClassifier()

### 검증셋으로 모델 성능 검증

트레인 셋에서 학습된 모델을   
검증 셋을 통해서 얼마나 성능이 나오는지 확인해보도록 하겠습니다.

In [20]:
X_val = val_data.drop(['index', 'target'], axis=1)  #validation 데이터에서 전처리된 문서 추출
y_val = val_data.target #validation 데이터에서 라벨 추출

In [21]:
y_pred = model.predict(X_val)
print(y_pred)

[0 0 1 ... 1 1 0]


In [126]:
from sklearn import metrics

print('RandomForestClassifier 의 예측 정확도는', round(metrics.accuracy_score(y_val, y_pred),3)) # 정확도 확인

RandomForestClassifier 의 예측 정확도는 0.846


아무런 튜닝을 하지 않았을 때 예측 정확도가 0.846이 나오네요.   
그럼 모델의 성능을 끌어올리기 위하여 랜덤 포레스트의 하이퍼 파라미터에 대해서 논의해보도록 하겠습니다.

## 랜덤 포레스트 하이퍼 파라미터 튜닝

랜덤 포레스트의 대표적인 하이퍼 파라미터 3개를 살펴보겠습니다. 

1. max_depth   

트리의 깊이로서, 랜덤 포레스트 트리의 루트 노드와 리프 노드 사이의 가장 긴 경로를 설정합니다.   
이 매개 변수를 사용하여 임의의 포리스트에있는 모든 트리가 성장할 수있는 깊이를 제한 할 수 있습니다.

일반적으로 트리를 너무 깊게 만들면 모델이 아주 복잡해지고 훈련데이터에 Overfitting되므로 적절히 설정해 주는 것이 중요합니다.

In [39]:
from sklearn.ensemble import RandomForestClassifier

model_md1 = RandomForestClassifier(max_depth = 1) # 모델을 객체에 할당
model_md10 = RandomForestClassifier(max_depth = 10)

model_md1.fit(X_train, y_train) # 모델 학습
model_md10.fit(X_train, y_train) 

pred_md1 = model_md1.predict(X_val)
pred_md10 = model_md10.predict(X_val)

print('RF max_depth = 1 의 예측 정확도는', round(metrics.accuracy_score(y_val, pred_md1),3)) # 정확도 확인
print('RF max_depth = 10 의 예측 정확도는', round(metrics.accuracy_score(y_val, pred_md10),3)) # 정확도 확인

RF max_depth = 1 의 예측 정확도는 0.746
RF max_depth = 10 의 예측 정확도는 0.85


2. n_estimators



랜덤 포레스트에서 결정 트리의 개수를 지정합니다.    
많이 설정할 수록 좋은 성능을 기대할 수 있지만, 계속 증가시킨다고 무조건 향상되는것은 아닙니다.   
또한 적을경우 과소적합이지만 증가에 따른 학습 수행시간 역시 오래걸리게 됩니다.

In [40]:
from sklearn.ensemble import RandomForestClassifier

model_ne1 = RandomForestClassifier(n_estimators = 1) # 모델을 객체에 할당
model_ne200 = RandomForestClassifier(n_estimators = 200)

model_ne1.fit(X_train, y_train) # 모델 학습
model_ne200.fit(X_train, y_train) 

pred_ne1 = model_ne1.predict(X_val)
pred_ne200 = model_ne200.predict(X_val)

print('RF n_estimators = 1 의 예측 정확도는', round(metrics.accuracy_score(y_val, pred_ne1),3)) # 정확도 확인
print('RF n_estimators = 200 의 예측 정확도는', round(metrics.accuracy_score(y_val, pred_ne200),3)) # 정확도 확인

RF n_estimators = 1 의 예측 정확도는 0.794
RF n_estimators = 200 의 예측 정확도는 0.85


3. max_features

전체에서 선택할 피처 개수 입니다.  
n_estimators는 클수록 트리의 깊이가 깊어지지만 max_features는 작을수록 트리가 깊어진다고 볼 수 있습니다.

In [147]:
from sklearn.ensemble import RandomForestClassifier

model_mf1 = RandomForestClassifier(max_features = 1) # 모델을 객체에 할당
model_mf3 = RandomForestClassifier(max_features = 3)

model_mf1.fit(X_train, y_train) # 모델 학습
model_mf3.fit(X_train, y_train) 

pred_mf1 = model_mf1.predict(X_val)
pred_mf3 = model_mf3.predict(X_val)

print('RF n_estimators = 1 의 예측 정확도는', round(metrics.accuracy_score(y_val, pred_mf1),3)) # 정확도 확인
print('RF n_estimators = 3 의 예측 정확도는', round(metrics.accuracy_score(y_val, pred_mf3),3)) # 정확도 확인

RF n_estimators = 1 의 예측 정확도는 0.842
RF n_estimators = 3 의 예측 정확도는 0.847


이렇게 하이퍼 파라미터를 비교하여 예측 정확도가 얼마나 차이가 나는지 확인 해 보았습니다.   
이제 적절히 하이퍼파리미터를 조절하여 최종 모델을 생성하여 예측 정확도를 도출해보겠습니다.

In [316]:
model_2 = RandomForestClassifier(max_depth = 10, n_estimators = 200, max_features = 3) # 최종 모델
model_2.fit(X_train, y_train) 
y_pred_2 = model_2.predict(X_val)
print(y_pred_2)
print('Tuned RandomForestClassifier 의 예측 정확도는', round(metrics.accuracy_score(y_val, y_pred_2),3)) # 정확도 확인

[0 0 0 ... 0 0 0]
Tuned RandomForestClassifier 의 예측 정확도는 0.86


최종 모델의 예측 정확도는 0.86이 나왔습니다.   
처음 튜닝하지 않은 모델 0.846에 비하여 정확도가 상승한 것을 확인해 볼 수 있습니다.

여러분들도 적절히 하이퍼 파라미터를 잘 튜닝하여 모델의 성능을 높여보세요 !

## test.csv 분류하기

이제 본격적으로 대회에서 주어진 정답이 없는 test 데이터의 라벨을 예측해보겠습니다.

전처리 모델 학습 단계에서 test 데이터를 사용하면 data leakage로 간주되니

test.csv는 반드시 모델 학습이 끝난 뒤 예측시에만 등장해야 함에 주의해주세요!

train.csv를 이용한 전처리/학습을 test.cvs에 적용하는 것은 허용되지만, test.csv를 전처리/모델링에 직접적으로 이용하면 실격입니다.

동일한 전처리 과정을 거치되, data leakage가 발생하지 않도록 주의합니다.

In [25]:
test = pd.read_csv('data/test.csv')
test = label_encoder(test, make_label_map(test))
test = test.drop('id', axis=1)
test.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country
0,47,0,304133,0,10,0,0,0,0,0,0,0,45,0
1,34,1,154227,0,10,1,1,1,0,0,0,0,75,0
2,31,2,158291,1,13,1,2,1,0,0,8614,0,40,0
3,28,0,183155,1,13,0,3,0,0,0,0,0,55,0
4,54,2,182543,0,10,2,4,2,0,1,0,0,40,1


트레인 셋과 검증 셋으로 나누어 성능을 확인해 준 것을 전체 train 데이터 셋을 이용하기 위하여

하나의 트레인 셋으로 통합해 다시 모델을 만들어주겠습니다.

In [26]:
X_train= data.drop(['target'], axis=1) #전체 training 데이터에서 독립변수 추출
y_train = data.target #전체 training 데이터에서 라벨 추출

앞서 튜닝한 하이퍼파라미터를 적용하여 모델을 생성합니다.

In [27]:
model = RandomForestClassifier(max_depth = 10, n_estimators = 200, max_features = 3) #앞서 튜닝한 하이퍼 파라미터 max_depth, n_estimators, max_features
model.fit(X_train, y_train) 
y_pred = model.predict(test)

## dacon 대회에 제출하기

이제 예측한 결과를 submission.csv 파일로 만들어서 대회 페이지에 제출해보도록 합시다.

제출한 뒤 리더보드를 통해 결과를 확인합시다.

In [28]:
# 제출용 Sample 파일을 불러옵니다
submission = pd.read_csv('data/sample_submission.csv')
submission.head()

,id,target
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [29]:
submission['target'] = y_pred

# 데이터가 잘 들어갔는지 확인합니다
submission

,id,target
0,0,0
1,1,0
2,2,1
3,3,1
4,4,0
...,...,...
15076,15076,1
15077,15077,1
15078,15078,0
15079,15079,0


submission을 csv 파일로 저장합니다.   
index=False란 추가적인 id를 부여할 필요가 없다는 뜻입니다.   
정확한 채점을 위해 꼭 index=False를 넣어주세요.   

In [30]:
submission.to_csv('submit_2.csv', index=False)

이렇게 생성된 submission.csv 파일을 데이콘 대회 페이지에 업로드 & 제출하여 결과를 확인해보세요!

문제를 해결하기 위한 여러분의 방법을 코드 공유 게시판에 공유해주세요

좋아요와 댓글을 합산하여 가장 높은 점수를 얻으신 분께 데이콘 후드가 제공됩니다!